# A specutils example for GBspec

This version uses the SpectrumList container. Could also consider a list of Spectrum1D or a SpectrumCollection.

This should reproduce Example 1 from the GBTIDL manual. The datafile you need is [here](http://safe.nrao.edu/wiki/pub/GB/Data/GBTIDLExampleAndSampleData/ngc5291.fits).

## SDFITS

We begin by disecting the typical SDFITS file, starting with raw plotting of a spectrum and some basic BINTABLE operations.


In [ ]:
%matplotlib inline


from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support
from specutils import Spectrum1D, SpectrumList

from astropy.nddata import StdDevUncertainty
from astropy.table import Table
from astropy.units import Unit
from astropy.wcs import WCS

from specutils.io import get_loaders_by_extension
from specutils.io.registers import data_loader
from specutils import Spectrum1D



In [ ]:
## A  few useful functions

In [ ]:
def my_stats(label,data,edge=0):
    """
    display mean,rms,min,max,npts
    also good for regression
    """
    if edge > 0:
        mean = data[edge:,-edge:].mean()
        rms  = data[edge:,-edge:].std()
        dmin = data[edge:,-edge:].min()
        dmax = data[edge:,-edge:].max()
    else:
        mean = data.mean()
        rms  = data.std()
        dmin = data.min()
        dmax = data.max()
    print("%s  %s %s %s %s %d" %  (label,repr(mean),repr(rms),repr(dmin),repr(dmax),len(data)-2*edge))

In [ ]:
def dcmeantsys(calon,caloff,tcal,mode=0):
    """
    following the GBTIDL routine with same name, get the tsys from the neighboring calon and caloff
    """
    nchan = len(calon)
    nedge = nchan // 10     # 10 %
    if mode == 0:
        meanoff = np.mean(caloff[nedge:-nedge])
        meandiff = np.mean(calon[nedge:-nedge] - caloff[nedge:-nedge])
        meanTsys = ( meanoff / meandiff * tcal + tcal/2.0 )
    else:
        meanTsys = np.mean( caloff[nedge:-nedge] / (calon[nedge:-nedge] - caloff[nedge:-nedge]) )
        meanTsys = meanTsys * tcal + tcal/2.0
    return meanTsys

And a worker routine, so we can benchmark some common operations on SD data

In [ ]:
def my_worker1(data,tsys):
    """ 
    an example of a simple ON/OFF for all rows, disregarding the all important PROCSEQN (see worker2)
    """
    (nrow,nchan) = data.shape
    print(data.shape)
    nrow2 = nrow//2
    data2 = np.zeros(nrow2*nchan).reshape(nrow2, nchan)
    for row in range(0,nrow,2):
        d1 = data[row]       # ON
        d0 = data[row+1]     # OFF
        data2[row//2] = tsys * (d1-d0) / d0
    return data2

In [ ]:
def my_worker2(data,tcal,nint=11,nscan=4):
    """ 
    an example of a PS style sequence of on/off:
    it takes 4 phases to create an integration
    The nint=11 should NOT be changed, this is for example1
    The nscan=4 can be changed to 2 and 1 if you want to test.
    """
    (nrow,nchan) = data.shape
    # reduction by 4:     2 from calon/caloff    2 from procseqn ON/OFF
    nrow4 = nrow * nscan // 4 // 4
    data4 = np.zeros(nrow4*nchan).reshape(nrow4, nchan)
    i=0
    for iscan in range(nscan):
        for iint in range(nint*2):
            i1 = iscan*nint*2*2*2 + iint*2   # calon    "ON"
            i2 = i1 + 1                      # caloff
            i3 = i1 + 2*nint*2               # calon    "OFF"
            i4 = i3 + 1                      # caloff
            ton1  = data[i1]
            toff1 = data[i2]
            tsys1 = dcmeantsys(ton1,toff1,tcal[i1])
            ton2  = data[i3]
            toff2 = data[i4]
            tsys2 = dcmeantsys(ton2,toff2,tcal[i3])
            #print(i,i1,i3,tsys1,tsys2)
            # review this math, i'm not getting the exact GBTIDL numbers out
            tsys = 0.5*(tsys1+tsys2)
            t1 = tsys * (toff1-toff2)/toff2
            t2 = tsys * (ton1-ton2)/ton2
            data4[i] = 0.5*(t1+t2)
            i=i+1
    return data4

## Input parameters

define the SDFITS file name and which row (0 being the first) we want to plot the spectrum of

In [ ]:
fname = 'ngc5291.fits'                         # 45MB    GBTIDL manual, ex1
#fname = 'AGBT15B_287_39.raw.vegas.A.fits'     # 386MB   EDGE (one of 6)
#fname = 'AGBT17B_151_01.raw.vegas.A.fits'     # 72MB    argus (one of 8)
row   = 0

Open the FITS file and point to the 2nd HDU, where the BINTABLE is located. No error checking. Data isn't really read yet.

In [ ]:
%%time
# 20ms: for ex1  fast because data is not really put in memory?
# 40ms for EDGE
nrow=0
try:
    hdu = fits.open(fname)
    header2 = hdu[1].header
    data2   = hdu[1].data
    nrow = len(data2)
    print("Found %d rows in %s" % (nrow,fname))
except:
    print("*** Error ***")

Lets stat all the numbers (nrow * nchan)

In [ ]:
%%time
# 790ms for ex1
my_stats(fname,data2[:]['DATA'])

Grab spectrum by row number. Get some stats (mean, rms, min, max)

In [ ]:
%%time 
#  3ms
flux  = data2[row]['DATA']  
nchan = len(flux)
chans = np.arange(nchan)
print("Found %d channels" % nchan)
#
my_stats('STATS for row %d:' % row,flux)

A super simple plot, channel number vs. flux.

In [ ]:
plt.plot(chans,flux)
plt.xlabel("Channel")
plt.ylabel("Flux");
plt.title("Raw Spectrum - row %d" % row);

This is a raw spectrum, mostly showing sky and bandpass.Hidden is a tiny signal, not even visible in the plot. For this we need to collect an "On" and "Off" spectrum and normalize this difference. 

### Columns and Rows in the BINTABLE

Apart from the 'DATA' column (which is the spectrum), there are lots of meta-data that we would like to keep. Most of these are scalars, so they can be retrieved as vectors of length **nrow**, e.g. the TCAL variable that is crucial for calibration.

In [ ]:
%%time 
# 17ms
tcal  = data2[:]['TCAL']  
rec   = np.arange(len(tcal))
print("TCAL mean/rms/min/max/ndata:")
my_stats('TCAL',tcal)

In [ ]:
plt.plot(rec,tcal);

Now a simple worker that performs an ON/OFF type operation on all columns. No data is returned, we just want to measure the speed. It will need the **tcal** column from the previous cell.

First a convenient numpy array data[nrow,nchans] that are the spectra (cf.waterfall plot)

In [ ]:
%%time
# 15 ms
spectra = data2[:]['DATA']

In [ ]:
%%time
#   0.004 empty call
#   185 ms just computing
#   214 ms stuffing it into an array (and returning it)
#   765 ms for EDGE
spectra2 = my_worker1(spectra, tcal)

In [ ]:
plt.plot(chans,spectra2[row]);

###  An example of the ON/OFF calibration

$$
     T_A = T_{sys} { { (ON-OFF)} \over {ON} }
$$

from GBTIDL's dcmeantsys.pro

    ;  mean_tsys = tcal * mean(nocal) / (mean(withcal-nocal)) + tcal/2.0
    
    
alternatively, one can also define **tsys** as function of spectrum, we try both below.

$$   
        T_{sys} = T_{cal}  { { (ON)} \over {ON-OFF} } + T_{cal}/2
$$

First, let's see how/where the data does the ON/OFF:

In [ ]:
ra = data2[:]['CRVAL2']  
dec = data2[:]['CRVAL3']
rec = range(len(ra))
plt.subplot(2,1,1)
plt.plot(rec,dec)
plt.subplot(2,1,2)
plt.plot(rec,ra);
my_stats('ra',ra)
my_stats('dec',dec)

Hence it is nodding in DEC only. Plus some very spurious deviations in RA, but small

In [ ]:
tcal1  = tcal[0]    # the first tcal in PROCSEQN=1
tcal2  = tcal[44]   # and in 2
print(tcal1,tcal2)  # they are the same, these are both XX
print(tcal[22])     # for the YY they are slightly different

In [ ]:
ton1 = spectra[0]
toff1 = spectra[1]
plt.plot(chans,ton1)
plt.plot(chans,toff1);

In [ ]:
tsys1 = tcal1 * toff1 / (ton1 - toff1) + tcal1/2.0
plt.plot(chans,tsys1)
my_stats('tsys1',tsys1[6000:12000])
# my_stats('tsys1',tsys1,nchan//10)

In [ ]:
tsys1 = dcmeantsys(ton1,toff1,tcal1,0)
print(tsys1)
ta1 = tsys1 * (ton1-toff1)/toff1
plt.plot(chans,ta1)
my_stats('ta1',ta1[6000:12000])

now we repeat this for PROCSEQ=2, so records 44 and 45 for the on/off (or will be be off/on ???)

In [ ]:
ton2 = spectra[44]
toff2 = spectra[45]
plt.plot(chans,ton2)
plt.plot(chans,toff2)
#
tsys2 = dcmeantsys(ton2,toff2,tcal2)
print(tsys2)

In [ ]:
tsys = (tsys1+tsys2)/2
print(tsys)
ta = tsys * (toff1-toff2)/toff2
plt.plot(chans,ta)
my_stats('ta',ta[6000:12001])

Now repeat this for all spectra via the specially handcrafted worker2 function:

In [ ]:
spectra4 = my_worker2(spectra,tcal,nscan=4)
print("Found %d spectra4" % len(spectra4))

ave = np.zeros(nchan)
for i in range(len(spectra4)):
    ave = ave + spectra4[i]
ave = ave / len(spectra4)
plt.plot(chans,ave)
plt.xlim(14500,18000)
plt.ylim(0.20,0.60)
my_stats('ave',ave[6000:12001])

In [ ]:
from astropy.convolution import convolve, Box1DKernel

ave2 = convolve(ave, Box1DKernel(51))
plt.plot(chans,ave,'b.')
plt.plot(chans,ave2,'r')
plt.xlim(14500,18000)
plt.ylim(0.20,0.60)

### A few words on row organization

All the magic of how calibration is done is in the organization and labeling of the rows. For the **position switching** of the NGC5271 data we will see 352 rows organized in a 5D matrix as follows:

    DATA(cal,int,sampler,procseqn,scan)
          2   11    2        2      4
          
with the following notes:

* DATA is labeled in column-major ("fortran") order, i.e. the first listed dimension runs fastest
* 2 * 11 * 2 * 2 * 4 = 352
* each procseqn also increments the scan number, so there are really 8 scan values but only 4 in that dimension
* each sampler is taken at the same time, so this means after 22 rows (cal * int) , time repeats in the next 
  sampler (the XX and YY pol in this case)
* in this data samplers point to different polarizations, in argus samplers are the beams/feeds for a single XX polarization, so the SAMPLER slot in the data can be used for different means
* for this type of PS data, the procseqn defines the ON (1) and the OFF (2) position
* Depending on observing modes, the dimensionality of the DATA is different, e.g. for nod data at GBT we see
         DATA(cal, int, pol, sampler, procseqn, scan)
  

### How about Pandas DataFrame's ?

A pandas dataframe could contain the whole BINTABLE, and with an attached 'engine' reduce the rows in the waterfall plot to lower dimension (eventually 1 if it's a single pointing), which is the desired result. 
Compare this with operators such as **np.sum(axis=1)/nrow** if you just want to get an average spectrum.

In [ ]:
from astropy.table import Table
import pandas

t= Table.read(fname,format='fits') 
if 'TDIM7' in t.colnames:
    print("Expect trouble, there is TDIM7")
t.meta
df = t.to_pandas()

In [ ]:
import pandas as pd
#import modin.pandas as pd
# import xray

x = np.arange(0,10)
x2 = x.reshape(5,2)
y = np.sqrt(x)
point = {'x': x2,    'y' : y}
points = 10 * [point]

#
df = pd.DataFrame().append(points)
df.x[4]

# so in other words, bintable to panda's manually is ok, but not via the Table interface

### Adding astropy units to the X and Y axis

specutils likes to work with things that have units. astropy to the rescue

In [ ]:
crval1 = data2[row]['CRVAL1']
cdelt1 = data2[row]['CDELT1']
crpix1 = data2[row]['CRPIX1']
freq0  = data2[row]['RESTFREQ']
freq = (crval1 + (np.arange(1,nchan+1) - crpix1) * cdelt1)/1e9
#
flux = flux * u.Unit("K")      # where is this in the header, we're assuming
freq = freq * u.Unit("GHz")    # or check CUNIT1 - usually not present in SDFITS files though

In [ ]:
# even though we've attached astropy units, they still work in matplotlib plotting
plt.plot(freq,flux)
plt.xlabel("Freq [%s]" % freq.unit)
plt.ylabel("Flux [%s]" % flux.unit);

### Creating a simple Spectrum1D object from specutils

In [ ]:
spec = Spectrum1D(spectral_axis=freq, flux=flux)

In [ ]:
f, ax = plt.subplots()  
ax.step(spec.spectral_axis, spec.flux);
# darn, the spectral axis still has 1e9 units, and it doesn't identify the units

## Designing our own reader in specutils

The specutils manual explains how to make your own reader. At the moment of writing there is no "sdfits" reader, so we're making a simple example with just a few meta-data. But this will simplify working with specutils.

In [ ]:
loaders = get_loaders_by_extension('fits')
print(loaders)

We need a list of FITS keywords and FIELD names from the BINTABLE that are going to be the meta-data
associated with each spectrum.


In [ ]:
# just a few important ones for now, there are about 70 in the full SDFITS

sdfits_headers = ['SCAN', 'PROCSEQN', 'CAL', 'OBJECT','SAMPLER', 'TCAL']

We are registering a special fits reader for the spectra, and use a SpectrumList.


In [ ]:
def identify_sdfits(origin, *args, **kwargs):
    print("IDENTIFY_SDFITS")
    try:
        with fits.open(args[0]) as hdulist:
            extname = hdulist[1].header['EXTNAME']
            if extname == 'SINGLE DISH':
                print("Hurray, we have SDFITS")
                return True
            else:
                print("Warning, skipping extname %s" % extname)
                return False
    except Exception:
        return Falseflux + sl2[i].flux

    

In [ ]:
@data_loader("sdfits", identifier=identify_sdfits, dtype=SpectrumList, extensions=['fits'])
def sdfits_loader(file_name, spectral_axis_unit=None, **kwargs):

    spectra = []
    with fits.open(file_name, **kwargs) as hdulist:
        header1= hdulist[0].header        
        header2= hdulist[1].header
        data   = hdulist[1].data
        nrow   = len(data)
        nchan  = 0
        for i in range(nrow):
            sp = data[i]['DATA']
            if nchan==0:
                nchan = len(sp)     # every spectrum in SDFITS has the same length
            crval1  = data[i]['CRVAL1']
            cdelt1  = data[i]['CDELT1']
            crpix1  = data[i]['CRPIX1']
            ctype1  = data[i]['CTYPE1']     # 'FREQ-OBS' to 'FREQ'; assuming SPECSYS='TOPOCENT'
            restfrq = data[i]['RESTFREQ']
            cunit1  = 'Hz'
            crval2  = data[i]['CRVAL2']
            crval3  = data[i]['CRVAL3']
            ctype2  = data[i]['CTYPE2']
            ctype3  = data[i]['CTYPE3']
            if ctype1 == 'FREQ-OBS': ctype1  = 'FREQ'
            wcs = WCS(header={'CDELT1': cdelt1, 'CRVAL1': crval1, 'CUNIT1': cunit1,
                              'CTYPE1': ctype1, 'CRPIX1': crpix1, 'RESTFRQ': restfrq,
                              'CTYPE2': ctype2, 'CRVAL2': crval2,
                              'CTYPE3': ctype3, 'CRVAL3': crval3})
                              

            meta = {}
            if False:
                # adding the actual FITS headers is for debugging, but not in production mode
                meta['header1'] = header1
                meta['header2'] = header2
            if True:
                for key in sdfits_headers:
                    if key in header1:
                        meta[key] = header1[key]
                    elif key in header2:
                        meta[key] = header2[key]
                    else:
                        meta[key] = data[i][key]    # why doesn't       key in data[i]    work?
                # add our row counter
                meta['_row'] = i
                        
        
            sp = sp * Unit('K')
            spec = Spectrum1D(flux=sp, wcs=wcs, meta=meta)
            spectra.append(spec)
            
    return  SpectrumList(spectra)

Now we are ready for some action! But lets define a small spectrum plotter with some nicer units



In [ ]:
def my_plot(sp, kms=False):
    """ plot a Spectrum1D 
    """
    if kms:
        x = sp.velocity   # doesn't work
    else:
        x = sp.spectral_axis / (1 * Unit("Hz"))/1e6
    if False:
        plt.plot(x, sp.flux)
        plt.xlabel('Frequency [MHz]')
        plt.ylabel('Flux [%s]' % sp.flux.unit)
    else:
        f, ax = plt.subplots()  # doctest: +IGNORE_OUTPUT
        ax.step(sp.spectral_axis, sp.flux) 

In [ ]:
%%time
# 3004 spectra in 7.5sec  
# 352 spectra in 931ms for ex1)
sl1 = SpectrumList.read(fname, format="sdfits")
nsp = len(sl1)
print("Found %d spectra" % nsp)

In [ ]:
print("Here are the first two spectra, the CalOn and CalOff")
for i in range(2):
    my_stats("spec    ",sl1[i].flux.value)
    my_stats("bintable",spectra[i])

In [ ]:
sp  = sl1[0]
x=sp.spectral_axis/(1 * Unit("Hz"))/1e6
y=sp.flux
type(y)
print(y.unit)


In [ ]:
# here is an issue:    cannot combine the ON and OFF for example if taken on different times

a=sl1[0]-sl1[1]
b=sl1[2]-sl1[3]
print(a)
print(b)
try:
    c = a + b
except:
    print("*** Cannot combine ***")
a1 = a / sl1[0]
print(a1)
# cannot do a+b, spectral axis is TOPO
# does this imply that if our ephemeris isn't good enough, they will not align in doppler space
# and thus refuse to be combined?

In [ ]:
# lets look at the meta data where we know there are changes
# there are 8 scans of 11 integrations each, two polarizations and the on/off cal cycle
for i in [0,1,22,44,88]:
    print(i,sl1[i].meta)


In [ ]:
%%time 
# 7.5s  (1.0s for ex1)
# this is like my_worker1() presented earlier
Spectra = []
for i in range(0,nsp,2):
    sp1 = sl1[i]      # this is the ON in PROCSEQN=1     
    sp2 = sl1[i+1]
    tsys = sl1[i].meta['TCAL']
    sp = (sp1-sp2)/sp2.flux.value * tsys     # don't use the units.....       
    sp.meta = sp1.meta
    #sp.flux *= tsys           # this is silly, can't modify the flux.....
    Spectra.append(sp)
    
sl2 =  SpectrumList(Spectra)   

Take the first calibrated spectrum, and see how it compares ith GBTIDL

In [ ]:
# GBTIDL:  getps,51,intnum=0  &  stats,/full
# 0.27865 0.16342 -1.5897 1.8587
my_stats("first R:",sl2[0].flux)
#my_plot(sl2[0])
plt.plot(chans,sl2[0].flux)
print(len(sl2))

In [ ]:
print(sp)
a = sp * 2
print(a)
a=sp.multiply(2*u.Unit('mJy'))   
print(sp)
print(a)
a = a * (2*u.Unit('mJy'))
print(a)


In [ ]:
for i in [0,1,11,22]:
    print(i,sl2[i].meta)

In [ ]:
from specutils.manipulation import (box_smooth, gaussian_smooth, trapezoid_smooth)
print(sp
    )
spec1 = Spectrum1D(flux=ave*u.Unit("K"), wcs=sp.wcs, meta=sp.meta)
spec1_bsmooth = box_smooth(spec1, width=3)
spec1_gsmooth = gaussian_smooth(spec1, stddev=3)
spec1_tsmooth = trapezoid_smooth(spec1, width=3)

In [ ]:
plt.plot(chans, spec1.flux) 
plt.xlim(14500,18000)
plt.ylim(0.2,0.6)

In [ ]:
plt.plot(chans, spec1_bsmooth.flux) 
plt.title("Boxcar Smooth")
plt.xlim(14500,18000)
plt.ylim(0.2,0.6)

In [ ]:
plt.plot(chans, spec1_gsmooth.flux)
plt.title("Gaussian Smooth")
plt.xlim(14500,18000)
plt.ylim(0.2,0.6)

In [ ]:
plt.plot(chans, spec1_tsmooth.flux) 
plt.title("Trapezoid Smooth")
plt.xlim(14500,18000)
plt.ylim(0.2,0.6)